Inelastic Relativistic Collision
https://www.feynmanlectures.caltech.edu/info/exercises/inelastic_relativistic_collision.html


A particle of mass m, moving at speed v = 4c/5, collides inelastically with a similar particle at rest.

(a) What is the speed vc of the composite particle?

(b) What is its mass mc?

In [3]:
using DynamicPolynomials
using JuMP
using MosekTools
using LinearAlgebra
using CSV
using DataFrames
import JSON
using Dates
using Gurobi
using FileIO
using JLD2

functions for generating monomials

In [13]:
function all_monomials_up_to_max_deg(x, deg)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:deg 
      for m=all_monomials_up_to_max_deg(x[2:end], deg)
    ]
    end
end

function mons_of_max_degree_and_unit(x, deg, u)
    [m
        for m=all_monomials_up_to_max_deg(x, deg)
        #if all(unit(m) .== u)
    ]
end
                
function degree_poly(p)
    maximum(degree.(monomials(p)))
end

function all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall) 
                for m=all_monomials_up_to_max_deg_overall(x[2:end], deg, deg_overall-k)
    ]
    end
end

function all_monomials_up_to_max_deg_overall_and_individual(x, deg, deg_overall, theDegrees)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall, theDegrees[1]) 
                for m=all_monomials_up_to_max_deg_overall_and_individual(x[2:end], deg, deg_overall-k, theDegrees[2:end])
    ]
    end
end

function mons_of_max_degree_and_unit_overall(x, deg, deg_overall, u)
    [m
        for m=all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
        #if all(unit(m) .== u)
    ]
end


mons_of_max_degree_and_unit_overall (generic function with 1 method)

In [5]:
@polyvar m_m m_c m_r v_m v_c c p_m p_c E_m E_r E_c   # the order matters per evaluation(?)

x =[m_m m_c m_r v_m v_c c p_m p_c E_m E_r E_c  ] 

1×11 Matrix{Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}:
 m_m  m_c  m_r  v_m  v_c  c  p_m  p_c  E_m  E_r  E_c

In [18]:
E_c = E_m + E_r  # conservation of energy
cEs =E_c^2 # conservation of energy squared 

E_r² + 2E_mE_r + E_m²

In [42]:
axioms= 
[
    #p_m^2*(c^2 - v_m^2)-m_m^2*v_m^2*c^2,
    p_m^2*(c^2-v_m^2) - m_m^2*v_m^2*c^2,    # momentum of moving particle p_m = m*v / (sqrt(1-v^2/c^2))
    E_m^2 - (m_m*c^2)^2 - (p_m*c)^2,       # square energy of moving particle
    E_r - m_r*c^2,                         # energey of particple at rest
    E_c^2 - (m_c *c)^2 - (p_c*c)^2,        # energy of composite particle
    #E_m + E_r - E_c,                       # conservation of energy
    2*E_m*E_r - E_c^2 + E_m^2 + E_r^2,     # conservation of energy squared 
    #cEs,  #2*E_m*E_r - E_c^2 + E_m^2 + E_r^2, # conservation of energy squared 
    #
    p_c - p_m,                             # conservation of momentum 
    v_m - 4/5*c,                          # moving velocity of the mass is 4/5c 
    m_r - m_m,                           # resting mass of particle at rest and particle at motion is equal 
    #m_c - m_m - m_r,                     # mass of composite particle
    p_c^2*(c^2-v_c^2) - m_c^2*v_c^2*c^2,  # momentum of the composite particle p_c = m_c*v_c / (sqrt(1-v^2/c^2))
]

9-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}}:
 c²p_m² - v_m²p_m² - m_m²v_m²c²
 E_m² - c²p_m² - m_m²c⁴
 E_r - m_rc²
 E_r² + 2.0E_mE_r + E_m² - c²p_c² - m_c²c²
 0.0
 p_c - p_m
 -0.8c + v_m
 m_r - m_m
 c²p_c² - v_c²p_c² - m_c²v_c²c²

In [ ]:
deg = 4
deg_overall = 8
theDegrees = [2 2 2 4 4 4 2 2 2 2 2] # m_m m_c m_r v_m v_c c p_m p_c E_m E_r E_c  

candidate_mons = [
    #mons_of_max_degree_and_unit_overall(x, deg, deg_overall, [])
    all_monomials_up_to_max_deg_overall_and_individual(x, deg, deg_overall, theDegrees)
    for ai=axioms
]
@show size.(candidate_mons)

# model = Model(Mosek.Optimizer)
model = Model(Gurobi.Optimizer)
mons_q = mons_of_max_degree_and_unit_overall([ m_c m_m p_m c ], deg, deg_overall, []) #  only include variables expected to appear in the final theorem 

coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="q")

q = sum(ci .* mi for (ci, mi)=zip(coeff_q, mons_q)) # Zip pairs things without needing a ref index, e.g., zip([1, 2, 3], [4,5,6])=((1,4), (2,5), (3,6))
coeff_αs = [
    @variable(model, [1:size(X,1)], base_name="α$i")
    for (i,X)=enumerate(candidate_mons)
    ]
@show size.(coeff_αs)
αs = [sum(ci .* mi) for (ci, mi)=zip(coeff_αs, candidate_mons)]

residual = q - sum(αᵢ * aᵢ for (αᵢ, aᵢ)=zip(αs,axioms));
eqs = coefficients(residual)

# Ensure that the sum of the coefficients on the terms involving m isn't zero, in order that m is part of expression
@constraint model sum(coeff_q[degree.(mons_q, [m_c ]).>0]) == 4.0 # insist that some variables will appear
#@constraint model sum(coeff_q[degree.(mons_q,[v_m]).>0]) == 0.0 # insist that some variables will not appear
@constraint model eqs .== 0
@objective model Max 0

optimize!(model)
@show termination_status(model)

In [53]:
value_poly = p -> sum(value.(coefficients(p)).* monomials(p))

#239 (generic function with 1 method)

In [54]:
value_q = value_poly(q)
value_q

-1.3333333333333333m_c⁴cs² + 5.333333333333332m_m²m_c²cs³ + 9.48148148148148m_m⁴cs⁴

In [27]:
@show value_αs = value_poly.(αs)

value_αs = value_poly.(αs) = Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}[11.111111111111114, 8.888888888888893cp_m² + 11.111111111111114v_mp_m² + 8.888888888888891m_m²c³ + 11.111111111111114m_m²v_mc²]


2-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}}:
 11.111111111111114
 8.888888888888893cp_m² + 11.111111111111114v_mp_m² + 8.888888888888891m_m²c³ + 11.111111111111114m_m²v_mc²

Solving the above for m_c^2 : 4/3 c^4 m_c^4 + 16/3 m_m^2 c^6 m_c^2 + 9.48 m_m^4 c^8


In [115]:
#p_c^2 * (c^2 - v_c^2) -  m_c^2 * v_c*2 * c^2             # momentum of composite particle
Solving the above for m_c^2 : 4/3 c^4 m_c^4 + 16/3 m_m^2 c^6 m_c^2 + 9.48 m_m^4 c^8


c²p_c² - v_c²p_c² - 2m_c²v_cc²

After reordering we get the correct composite mass expression:
